# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/jack/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/decl-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/

In [3]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'N'
vuln = [False, False]  # fist element is NS, second element is EW

hands = ['87.84.T7.AKJ9764','JT53.QT.AJ6542.8','KQ96.K9.Q83.QT32','A42.AJ76532.K9.5']

auction = ['3C', 'PASS', '4C', '4H', 'PASS', 'PASS', 'PASS']

play = ['CA', 'C8', 'C3', 'C5',
    'S8', 'ST', 'SQ', 'SA',
    'DK', 'DT', 'D6', 'D3',
    'D9', 'D7', 'DA', 'D8', 
    'DJ', 'DQ', 'HJ', 'S7']

In [4]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler, False)

In [5]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

await card_by_card.analyze()

analyzing the bidding
3C OK NN-value: 0.942


Loaded lib dds.dll


PASS OK NN-value: 0.999
4C Suggested bid from NN: CandidateBid(bid=PASS, insta_score=0.9221, expected_score=-51.20, adjust=  46)
4C is not in the bids from the neural network
4H OK NN-value: 0.454
PASS OK NN-value: 0.914
PASS OK NN-value: 0.903
PASS Suggested bid from NN: CandidateBid(bid=5C  , insta_score=0.6373, expected_score=-265.75, adjust=  32)
PASS NN-values:CandidateBid(bid=PASS, insta_score=0.3227, expected_score=-360.40, adjust=  16)
analyzing opening lead
CA
CA OK
analyzing play
C8 OK
C3 OK
C5 OK
S8 OK
ST OK
SQ OK
SA ? losing: 0.40
DK OK
DT OK
D6 OK
D3 OK
D9 OK
D7 OK
DA OK
D8 OK
DJ OK
DQ OK
HJ OK
S7 OK


In [6]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

{'bid': '3C',
 'who': 'Analysis',
 'candidates': [{'call': '3C', 'insta_score': 0.942}],
 'hcp': [3.8, 3.6, 3.5, 1.9, 3.6, 3.6, 3.6, 2.1, 3.8, 3.5, 3.6, 2.0],
 'shape': [10.7, 10.6, 10.7]}

the engine very confidently opens `1C` and doesn't even consider `1N`

In [7]:
# what about the opening lead? let's see...

card_by_card.cards['CA'].to_dict()

{'card': 'CA',
 'quality': 'Good',
 'hcp': [4.2, 3.0, 4.0, 1.8, 3.6, 2.4, 4.0, 3.1, 3.3, 5.8, 2.9, 0.9],
 'shape': [8.1, 8.5, 14.0],
 'candidates': [{'card': 'DT',
   'insta_score': 0.154,
   'expected_tricks_sd': 12.7,
   'p_make_contract': 0.0},
  {'card': 'CK',
   'insta_score': 0.623,
   'expected_tricks_sd': 12.8,
   'p_make_contract': 0.0},
  {'card': 'Sx',
   'insta_score': 0.061,
   'expected_tricks_sd': 12.8,
   'p_make_contract': 0.0}],
 'samples': ['8x.8x.Tx.AKJ9xxx KTx.xxxx.QJxx.T8 A9xxx..8xxx.Qxxx QJx.AKQJT9x.AK9. 0.63183',
  '8x.8x.Tx.AKJ9xxx AKJx.Qxx.9xxx.Qx T9xxx..KQJx.T8xx Qx.AKJT9xxx.A8x. 0.61970',
  '8x.8x.Tx.AKJ9xxx ATx.Jxx.QJxxx.Q8 QJxxx..K9xx.Txxx K9x.AKQT9xxx.A8. 0.61017',
  '8x.8x.Tx.AKJ9xxx AKT9xx.Txx.xx.8x xx.x.KJ98xx.QTxx QJx.AKQJ9xx.AQx. 0.60829',
  '8x.8x.Tx.AKJ9xxx AT9xxx.Txx.Jxx.8 Kxx.x.Q98x.QTxxx QJ.AKQJ9xx.AKxx. 0.60587',
  '8x.8x.Tx.AKJ9xxx KTxxxx.Jxx.J9.Qx Q9x.Q.Kxxxx.T8xx AJ.AKT9xxx.AQ8x. 0.60360',
  '8x.8x.Tx.AKJ9xxx AJxx.AJx.98xx.8x Qxxx..QJxxx.QTx

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [8]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

{'card': 'SJ',
 'quality': 'Bad',
 'hcp': [8.0, 11.0],
 'shape': [2.1, 1.9, 2.3, 6.8, 3.6, 3.0, 3.1, 3.2],
 'candidates': [{'card': 'SJ',
   'insta_score': 0.203,
   'expected_tricks_dd': 10.2,
   'p_make_contract': 1.0,
   'expected_score_dd': 426},
  {'card': 'S3',
   'insta_score': 0.797,
   'expected_tricks_dd': 9.2,
   'p_make_contract': 0.2,
   'expected_score_dd': 44},
  {'card': 'S5',
   'insta_score': 0.797,
   'expected_tricks_dd': 9.2,
   'p_make_contract': 0.2,
   'expected_score_dd': 44}],
 'samples': ['8x.K98.Q.AQT9xxx JTxx.QT.AJxxxx.8 KQ9x.x.T8xx.KJxx Axx.AJxxxxx.K9.x 0.12589',
  '98x.K9x..AQJT9xx JTxx.QT.AJxxxx.8 KQx.8.QT8xx.Kxxx Axx.AJxxxxx.K9.x 0.05880',
  '8x.x.T8x.AT9xxxx JTxx.QT.AJxxxx.8 KQ9x.K98.Qx.KQJx Axx.AJxxxxx.K9.x 0.05868',
  '8x.K98.T.AQJT9xx JTxx.QT.AJxxxx.8 KQ9x.x.Q8xx.Kxxx Axx.AJxxxxx.K9.x 0.05575',
  '8x.K8x.x.AQJT9xx JTxx.QT.AJxxxx.8 KQ9x.9.QT8x.Kxxx Axx.AJxxxxx.K9.x 0.05550']}

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)